In [3]:
import os
import sys
import numpy as np
from pprint import pprint
from tqdm import tqdm
import pandas as pd
from datetime import datetime
from importlib import reload
import math
from IPython.display import display, Markdown
from datetime import datetime

sys.path.insert(0, os.path.abspath('../tesi_sabella'))
# pd.options.display.max_rows = 999

import pyfluxc.pyfluxc as flux
import data_generator as generator
reload(flux)
reload(generator)

<module 'data_generator' from '/Users/swedishhdcamera/Desktop/Projects/tesi_sabella/tesi_sabella/data_generator.py'>

In [4]:
# Constants ----- #
BUCKET = 'CIC_IDS_2017_Monday'
PULL_EVERY_X_MINUTES = 15
PULL_DELTA_TIME = pd.Timedelta(minutes=PULL_EVERY_X_MINUTES)

TCPREPLAY_START = {
    "Monday": "2020-04-04T17:19:06.730097",
    "Tuesday": "2020-04-05T14:09:30.933025",
    "Wednesday": "2020-04-06T08:46:54.338920",
    "Thursday": "2020-04-07T08:08:37.127602808",
    "Friday": "2020-04-07T22:41:05.814784058"}
TCPREPLAY_START = { k: pd.to_datetime(s, format='%Y-%m-%dT%H:%M:%S.%f') for k, s in TCPREPLAY_START.items() }

CIC_IDS_2017_START = {
    "Monday": "2017-07-03T08:55:58.598308",
    "Tuesday": "2017-07-04T08:53:32.364079",
    "Wednesday": "2017-07-05T08:42:42.084372",
    "Thursday": "2017-07-06T08:58:58.492265",
    "Friday": "2017-07-07T08:59:39.599128"}
CIC_IDS_2017_START = { k: pd.to_datetime(s, format='%Y-%m-%dT%H:%M:%S.%f') for k, s in CIC_IDS_2017_START.items() }

In [5]:
# Functions ----- #
def mprint(s):
    display(Markdown(s))

# Generating time series

We pull only the time range in which samples are available

In [6]:
fclient = flux.FluxClient(); 
start, end = fclient.bucket_timerange(BUCKET)
print(f"first sample: {start}\nlast sample: {end}\nTotal: {end-start}")

first sample: 2020-04-04 17:18:30
last sample: 2020-04-05 08:48:55
Total: 0 days 15:30:25


In [7]:
cicids2017 = generator.CICIDS2017(BUCKET, '15s', fclient, start)

Extract data from influxdb in time intervals due to influx kill signal

In [8]:
i = 1
pull_start = start
ts = []
while True:
    pull_end = pull_start + PULL_DELTA_TIME
    
    print("------------")
    print(f"first sample: {pull_start}\nlast sample: {pull_end}\nTotal: {pull_end-pull_start}")
    print("------------")
    
    # TODO: return tuple
    t_now = datetime.now()
    pull_start, new_samples = cicids2017.pull(stop=pull_end)
    ts.append(datetime.now() - t_now)
    if new_samples is None:
        break

mprint(f"**Mean pulling time:** {np.mean(ts)}")

------------
first sample: 2020-04-04 17:18:30
last sample: 2020-04-04 17:33:30
Total: 0 days 00:15:00
------------
------------
first sample: 2020-04-04 17:33:30
last sample: 2020-04-04 17:48:30
Total: 0 days 00:15:00
------------
------------
first sample: 2020-04-04 17:48:30
last sample: 2020-04-04 18:03:30
Total: 0 days 00:15:00
------------
------------
first sample: 2020-04-04 18:03:30
last sample: 2020-04-04 18:18:30
Total: 0 days 00:15:00
------------
------------
first sample: 2020-04-04 18:18:30
last sample: 2020-04-04 18:33:30
Total: 0 days 00:15:00
------------
------------
first sample: 2020-04-04 18:33:30
last sample: 2020-04-04 18:48:30
Total: 0 days 00:15:00
------------
------------
first sample: 2020-04-04 18:48:30
last sample: 2020-04-04 19:03:30
Total: 0 days 00:15:00
------------
------------
first sample: 2020-04-04 19:03:30
last sample: 2020-04-04 19:18:30
Total: 0 days 00:15:00
------------
------------
first sample: 2020-04-04 19:18:30
last sample: 2020-04-04 1

KeyboardInterrupt: 

In [ ]:
DIFF = False
cicids2017_df = cicids2017.to_pandas(DIFF)

# Setting correct timing before saving
day = BUCKET.split("_")[-1]
delta = TCPREPLAY_START[day] - CIC_IDS_2017_START[day]
new_delta_time = (cicids2017_df.index.levels[2] - delta)
cicids2017_df.index.set_levels(new_delta_time, level="_time", inplace=True)

cicids2017_df.to_pickle(f"./{BUCKET}{'_no_delta' if not DIFF else ''}.pkl")

In [ ]:
def cest_to_utc(x):
    return x.tz_localize(tz='CET').tz_convert('UTC').tz_localize(tz=None)
    
TCPREPLAY_START_UTC = { k: cest_to_utc(x) for k, x in TCPREPLAY_START.items() }
for k, v in TCPREPLAY_START_UTC.items():
    print(v.strftime(format='%Y-%m-%dT%H:%M:%S.%f'))

In [10]:
df = cicids2017.to_pandas(False)

In [30]:
ndpi_rcvd = [x for x in list(df.columns) if "ndpi:bytes_rcvd" in x and "unknown" not in x]
ndpi = df.loc["pc", "192.168.10.12"][ndpi_rcvd].sum(axis=1)
bytes_rcvd = df.loc["pc", "192.168.10.12"]["traffic:bytes_rcvd"]
unknown = df.loc["pc", "192.168.10.12"]["ndpi:bytes_rcvd__unknown"]
pd.concat([bytes_rcvd-ndpi, unknown], axis=1)

/Users/swedishhdcamera/Desktop/Projects/tesi_sabella/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: PerformanceWarning: indexing past lexsort depth may impact performance.
  
/Users/swedishhdcamera/Desktop/Projects/tesi_sabella/venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: PerformanceWarning: indexing past lexsort depth may impact performance.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/swedishhdcamera/Desktop/Projects/tesi_sabella/venv/lib/python3.7/site-packages/ipykernel_launcher.py:4: PerformanceWarning: indexing past lexsort depth may impact performance.
  after removing the cwd from sys.path.


,0,ndpi:bytes_rcvd__unknown
_time,,
2020-04-04 17:20:45,2.423000e+03,19420.0
2020-04-04 17:21:00,4.040600e+04,19420.0
2020-04-04 17:21:15,4.040600e+04,19420.0
2020-04-04 17:21:30,4.040600e+04,19420.0
2020-04-04 17:21:45,5.599200e+04,29307.0
...,...,...
2020-04-04 20:02:30,2.126915e+07,206214.0
2020-04-04 20:02:45,2.126921e+07,206214.0
2020-04-04 20:03:00,2.126921e+07,206214.0


In [29]:
ndpi_rcvd

['ndpi:bytes_rcvd__rpc',
 'ndpi:bytes_rcvd__video-media',
 'ndpi:bytes_rcvd__streaming',
 'ndpi:bytes_rcvd__network_file_system',
 'ndpi:bytes_rcvd__p2p-iptv',
 'ndpi:bytes_rcvd__aaa-protocol',
 'ndpi:bytes_rcvd__maps',
 'ndpi:bytes_rcvd__other',
 'ndpi:bytes_rcvd__iot',
 'ndpi:bytes_rcvd__database',
 'ndpi:bytes_rcvd__dns',
 'ndpi:bytes_rcvd__update',
 'ndpi:bytes_rcvd__search-engine',
 'ndpi:bytes_rcvd__mail_service',
 'ndpi:bytes_rcvd__tunneling',
 'ndpi:bytes_rcvd__voip',
 'ndpi:bytes_rcvd__instant-message-protocol',
 'ndpi:bytes_rcvd__video_file',
 'ndpi:bytes_rcvd__sysadmin',
 'ndpi:bytes_rcvd__monitoring',
 'ndpi:bytes_rcvd__http',
 'ndpi:bytes_rcvd__cloud storage',
 'ndpi:bytes_rcvd__p2p_file_sharing',
 'ndpi:bytes_rcvd__online_encyclopedia',
 'ndpi:bytes_rcvd__audio_file',
 'ndpi:bytes_rcvd__chat',
 'ndpi:bytes_rcvd__mining',
 'ndpi:bytes_rcvd__device discovery',
 'ndpi:bytes_rcvd__other-l4',
 'ndpi:bytes_rcvd__music-service',
 'ndpi:bytes_rcvd__message-broker',
 'ndpi:bytes_r